The goal of this notebook is to code a decision tree classifier that can be used with the following API.

# Import Statements

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
color = sns.color_palette()
sns.set_style('darkgrid')

# Load and Prepare Data

#### Format of the data
- last column of the data frame must contain the label and it must also be called "label"
- there should be no missing values in the data frame

In [ ]:
df = pd.read_csv("data/Iris.csv")
df = df.drop("Id", axis=1)
df = df.rename(columns={"species": "label"})

In [ ]:
df.head()

# Train-Test-Split

In [ ]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [ ]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

# Helper Functions

In [ ]:
data = train_df.values
data[:5]

### Data pure?

In [ ]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [ ]:
check_purity(train_df[train_df.petal_width <=0.8].values)

In [ ]:
check_purity(train_df[train_df.petal_width >0.8].values)

In [ ]:
sns.lmplot(data=df, x = 'petal_width', y='petal_length', hue='label', 
           fit_reg=False, size = 6,  aspect = 1.5);
plt.vlines(x=0.8, ymin=1, ymax=7);

### Classify

In [ ]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [ ]:
classify_data(train_df[train_df.petal_width <=0.8].values)

In [ ]:
classify_data(train_df[train_df.petal_width >0.8].values)

In [ ]:
classify_data(train_df[train_df.petal_width >1.6].values)

### Potential splits?

In [ ]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [ ]:
potential_splits = get_potential_splits(train_df.values)

In [ ]:
sns.lmplot(data=train_df, x = 'petal_width', y='petal_length', hue='label', 
           fit_reg=False, size = 6,  aspect = 1.5)
plt.vlines(x = potential_splits[3], ymin=1, ymax=7);
#plt.hlines(xmin=0, xmax=2.5, y = potential_splits[2]);

### Split Data

In [ ]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

In [ ]:
split_column = 3
split_value = 0.8
data_below, data_above = split_data(train_df.values, split_column, split_value)

In [ ]:
plotting_df = pd.DataFrame(train_df.values , columns=df.columns)
#plotting_df = pd.DataFrame(data_below , columns=df.columns)
#plotting_df = pd.DataFrame(data_above , columns=df.columns)

sns.lmplot(data = plotting_df, x = 'petal_width', y='petal_length', hue='label', 
           fit_reg=False, size = 6,  aspect = 1.5);
plt.vlines(x = split_value, ymin=1, ymax=7);
plt.xlim(0,2.6);

### Lowest Overall Entropy?

In [ ]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [ ]:
calculate_entropy(data_below)

In [ ]:
calculate_entropy(data_above)

In [ ]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [ ]:
calculate_overall_entropy(data_below, data_above)

In [ ]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [ ]:
determine_best_split(train_df.values, potential_splits)

# Decision Tree Algorithm

### Representation of the Decision Tree

In [ ]:
example_tree = {"petal_width <= 0.8": ["Iris-setosa", 
                                       {"petal_width <= 1.65": [{"petal_length <= 4.9": ["Iris-versicolor", 
                                                                                         "Iris-virginica"]}, 
                                                                "Iris-virginica"]}]}

### Algorithm

In [ ]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [ ]:
tree = decision_tree_algorithm(train_df[train_df.label != 'Iris-virginica'])
pprint(tree)

In [ ]:
tree = decision_tree_algorithm(train_df)
pprint(tree)

In [ ]:
### Pruning Tree
tree = decision_tree_algorithm(train_df, min_samples = 60)
pprint(tree)

In [ ]:
### Max Depth
tree = decision_tree_algorithm(train_df, max_depth= 3)
pprint(tree)

In [ ]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

# Classification

In [ ]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [ ]:
tree

In [ ]:
example = test_df.iloc[0, :4]
example

In [ ]:
classify_example(example, tree)

# Calculate Accuracy

In [ ]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [ ]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

In [ ]:
test_df.head()

In [ ]:
test_df[test_df.classification_correct == False]

In [ ]:
tree